In [1]:
# Import các thư viện và cài đặt chrome options
import re
import requests
from bs4 import BeautifulSoup
import csv
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from tqdm.auto import tqdm
import time
import datetime
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
main_category = input("Nhập main_category: ")
sub_category = input("Nhập sub_category: ")

In [3]:
# Tạo chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Chạy ẩn danh
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")


In [4]:
# Khởi tạo trình duyệt WebDriver
driver = webdriver.Chrome(options=chrome_options)

# Khai báo các danh sách để lưu thông tin sản phẩm
product_names = []
prices = []
links = []
ratings = []
reviews = []

In [5]:
# URL của trang tìm kiếm trên Amazon
url = f'https://www.amazon.com/s?i=specialty-aps&bbn=16225007011&rh=n%3A16225007011%2Cn%3A172456&ref=nav_em__nav_desktop_sa_intl_computer_accessories_and_peripherals_0_2_6_2'

In [6]:
# Hàm cào dữ liệu khi mỗi sản phẩm được hiển thị trên một hàng
def findasrow(url):
    for i in tqdm(range(1, 4), desc="Data From Amazon"):
        driver.get(url)
        time.sleep(5)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = soup.find_all('div', 'a-section a-spacing-small a-spacing-top-small')
        temp = 0
        for div in divs:
            if temp == 0:
                temp += 1
                continue
            product = div.find('a', 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
            if product is None:
                product_names.append(None)
            else:
                product_names.append(product.text)

            link = div.find('a', 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
            if link is None:
                links.append(None)
            else:
                links.append("https://www.amazon.com/" + link.get('href'))

            price = div.find('span', 'a-price-whole')
            if price is None:
                prices.append(None)
            else:
                prices.append(price.text)

            rate = div.find('span', 'a-icon-alt')
            if rate is None:
                if link is None:
                    pass
                ratings.append(None)
            else:
                ratings.append(rate.text.split()[0])

            review = div.find('span', 'a-size-base s-underline-text')
            if review is None:
                if link is None:
                    pass
                reviews.append(None)
            else:
                reviews.append(review.text)

        divs_pagination = soup.find('div', 'a-section a-text-center s-pagination-container')
        if not divs_pagination:
            print(" EComplete get data")
            break
        else:
            next_page_link = divs_pagination.find('a', 's-pagination-item s-pagination-next s-pagination-button s-pagination-separator')
            if not next_page_link:
                print("Completed")
                break
            else:
                url = "https://www.amazon.com/" + next_page_link.get('href')

    # Lưu dữ liệu vào file CSV
    print("Length of products: ", len(product_names))
    print("Length of prices: ", len(prices))
    print("Length of links: ", len(links))
    print("Length of ratings: ", len(ratings))
    print("Length of reviews: ", len(reviews))

    filename = f"part1_data_{main_category}_{sub_category}.csv"
    with open(filename, 'w', newline="", encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['MainCategory', 'SubCategory', 'Product Name', 'Price', 'Link', 'Rating', 'Review'])
            for product, price, link, rate, review in zip(product_names, prices, links, ratings, reviews):
                csv_writer.writerow([main_category, sub_category, product, price, link, rate, review])

    print(f"SAVED THE FILE!!! part1_data_{main_category}_{sub_category}.csv")

In [7]:
# Hàm cào dữ liệu khi mỗi sản phẩm được hiển thị trong một ô
def crawlastiles(url):
    for i in tqdm(range(1, 41), desc="Data From Amazon"):
        driver.get(url)
        #time.sleep(8)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = soup.find_all('div', 'sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20')
        temp = 0
        for div in divs:
            if temp == 0:
                temp += 1
                continue
            product = div.find('a', 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
            if product is None:
                product_names.append(None)
            else:
                product_names.append(product.text)
            
            link = div.find('a', 'a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal')
            if link is None:
                links.append(None)
            else:
                links.append("https://www.amazon.com/" + link.get('href'))
            
            price = div.find('span', 'a-price-whole')
            if price is None:
                prices.append(None)
            else:
                prices.append(price.text)
            
            rate = div.find('span', 'a-icon-alt')
            if rate is None:
                if link is None:
                    pass
                ratings.append(None)
            else:
                ratings.append(rate.text.split()[0])
            
            reviewnumber = div.find('span','a-size-base s-underline-text')
            if reviewnumber is None:
                if link is None:
                    pass
                reviews.append(None)
            else:
                reviews.append(reviewnumber.text)

        divs_pagination = soup.find('div', 'a-section a-text-center s-pagination-container')
        if not divs_pagination:
            print("Complete get data")
            break
        else:
            next_page_link = divs_pagination.find('a','s-pagination-item s-pagination-next s-pagination-button s-pagination-separator')
            if not next_page_link:
                print("Completed, end of pagination")
                break
            else:
                url = "https://www.amazon.com" + next_page_link.get('href')
                driver.get(url)

    # Lưu dữ liệu vào file CSV
    print("Length of products: ", len(product_names))
    print("Length of prices: ", len(prices))
    print("Length of links: ", len(links))
    print("Length of ratings: ", len(ratings))
    print("Length of reviews: ", len(reviews))
    
    filename = f"part1_data_{main_category}_{sub_category}.csv"
    with open(filename, 'w', newline="", encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerow(['MainCategory', 'SubCategory', 'Product Name', 'Price', 'Link', 'Rating', 'Review'])
            for product, price, link, rate, review in zip(product_names, prices, links, ratings, reviews):
                csv_writer.writerow([main_category, sub_category, product, price, link, rate, review])

    print(f"SAVED THE FILE!!! part1_data_{main_category}_{sub_category}.csv")

In [8]:
# Kiểm tra cách trình bày sản phẩm để quyết định cách thực hiện việc cào dữ liệu
driver.get(url)

soup = BeautifulSoup(driver.page_source, 'html.parser')
divs = soup.find_all('div', 'sg-col-4-of-24 sg-col-4-of-12 s-result-item s-asin sg-col-4-of-16 sg-col s-widget-spacing-small sg-col-4-of-20')

if not divs:
    print("Đang thực hiện crawl theo hàng sản phẩm()...")
    findasrow(url)  # Truyền url vào hàm findasrow()
else:
    print("Đang thực hiện crawl theo ô sản phẩm ()...")
    crawlastiles(url)  # Truyền url vào hàm crawlastiles()


Đang thực hiện crawl theo ô sản phẩm ()...


Data From Amazon:   0%|          | 0/40 [00:01<?, ?it/s]

Complete get data
Length of products:  0
Length of prices:  0
Length of links:  0
Length of ratings:  0
Length of reviews:  0
SAVED THE FILE!!! part1_data_Computer_Computer Accessories & Peripherals.csv


#PHẦN 2: CÀO CHI TIẾT SẢN PHẨM


In [9]:
def calculate_shipping_days(estimated_delivery_date_str, crawl_date_str):
    try:
        estimated_delivery_date = datetime.datetime.strptime(estimated_delivery_date_str, '%A, %B %d')
        crawl_date = datetime.datetime.strptime(crawl_date_str, '%A, %B %d')
        return (estimated_delivery_date - crawl_date).days
    except ValueError:
        return 'N/A'

In [10]:
def clean_text(text):
    return re.sub(r'[\u200e]', '', text)

In [11]:
def detailproduct_bucket(soup, divs2I):
    dimension = asinid = datefirstavailable = manufactures = country = sellerrank = color = modelnumber = weight = price = priceamzship = eic = shipping_days = 'None'

    divs3 = divs2I.find_all('span', class_='a-text-bold')
    for div in divs3:
        if 'Package Dimensions' in div.text.strip() or 'Dimensions' in div.text.strip():
            div4 = div.find_next_sibling('span')
            dimension = clean_text(div4.text.strip() if div4 else 'None')
        elif 'Item model number' in div.text.strip():
            div4 = div.find_next_sibling('span')
            modelnumber = clean_text(div4.text.strip() if div4 else 'None')
        elif 'Date First Available' in div.text.strip():
            div4 = div.find_next_sibling('span')
            datefirstavailable = clean_text(div4.text.strip() if div4 else 'None')
        elif 'Manufacturer' in div.text.strip():
            div4 = div.find_next_sibling('span')
            manufactures = clean_text(div4.text.strip() if div4 else 'None')
        elif 'ASIN' in div.text.strip():
            div4 = div.find_next_sibling('span')
            asinid = clean_text(div4.text.strip() if div4 else 'None')
        elif 'Country of Origin' in div.text.strip():
            div4 = div.find_next_sibling('span')
            country = clean_text(div4.text.strip() if div4 else 'None')
        elif 'Best Sellers Rank' in div.text.strip():
            parent_tag = div.parent
            sellerrank = clean_text(parent_tag.get_text(strip=True))
        elif 'Color' in div.text.strip():
            div4 = div.find_next_sibling('span')
            color = clean_text(div4.text.strip() if div4 else 'Not Given')

    divs2Ia = soup.find('div', class_='a-popover-preload', id='a-popover-agShipMsgPopover')
    if (divs2Ia):
        table = divs2Ia.find('table', class_='a-lineitem')
        if (table):
            lines = table.find_all('td', class_='a-span9 a-text-left')
            for td in lines:
                if 'Price' in td.text:
                    divsprice = td.find_next_sibling('td', class_='a-span2 a-text-right')
                    price = clean_text(divsprice.text.strip() if divsprice else 'None')
                if 'AmazonGlobal Shipping' in td.text:
                    divsamzship = td.find_next_sibling('td', class_='a-span2 a-text-right')
                    priceamzship = clean_text(divsamzship.text.strip() if divsamzship else 'None')
                if 'Estimated Import Charges' in td.text:
                    diveic = td.find_next_sibling('td', class_='a-span2 a-text-right')
                    eic = clean_text(diveic.text.strip() if diveic else 'None')

    estimated_delivery_date_tag = soup.find('span', class_='a-text-bold')
    estimated_delivery_date = clean_text(estimated_delivery_date_tag.text.strip() if estimated_delivery_date_tag else 'N/A')
    crawl_date = datetime.datetime.now().strftime('%A, %B %d')
    shipping_days = calculate_shipping_days(estimated_delivery_date, crawl_date)
    
    return {
        "Dimension": dimension,
        "ASIN": asinid,
        "Date First Available": datefirstavailable,
        "Manufacturer": manufactures,
        "Country of Origin": country,
        "Best Sellers Rank": sellerrank,
        "Color": color,
        "Item Model Number": modelnumber,
        "Item Weight": weight,
        "Price": price,
        "AmazonGlobal Shipping": priceamzship,
        "Estimated Import Charges": eic,
        "Day Delivered": str(shipping_days)
    }

In [12]:
def detailproduct_table(soup, divs2):
    dimension = asinid = datefirstavailable = manufactures = country = sellerrank = color = modelnumber = weight = price = priceamzship = eic = shipping_days = 'None'

    divs3 = divs2.find_all('th', class_='a-color-secondary a-size-base prodDetSectionEntry')
    for div in divs3:
        if 'Manufacturer' in div.text.strip():
            divs7 = div.find_next_sibling('td', class_='a-size-base prodDetAttrValue')
            manufactures = clean_text(divs7.text.strip() if divs7 else 'None')
        if 'Dimensions' in div.text.strip():
            divs4 = div.find_next_sibling('td', class_='a-size-base prodDetAttrValue')
            dimension = clean_text(divs4.text.strip() if divs4 else 'None')
        if 'ASIN' in div.text.strip():
            divs5 = div.find_next_sibling('td', class_='a-size-base prodDetAttrValue')
            asinid = clean_text(divs5.text.strip() if divs5 else 'None')
        if 'Date First Available' in div.text.strip():
            divs6 = div.find_next_sibling('td', class_='a-size-base prodDetAttrValue')
            datefirstavailable = clean_text(divs6.text.strip() if divs6 else 'None')
        if 'Country of Origin' in div.text.strip():
            divs8 = div.find_next_sibling('td', class_='a-size-base prodDetAttrValue')
            country = clean_text(divs8.text.strip() if divs8 else 'None')
        if 'Best Sellers Rank' in div.text.strip():
            divs9 = div.find_next_sibling('td')
            sellerrank = clean_text(divs9.text.strip() if divs9 else 'None')
        if 'Color' in div.text.strip():
            divs10 = div.find_next_sibling('td', class_='a-size-base prodDetAttrValue')
            color = clean_text(divs10.text.strip() if divs10 else 'Not Given')
        if 'Item model number' in div.text.strip():
            divs11 = div.find_next_sibling('td', class_='a-size-base prodDetAttrValue')
            modelnumber = clean_text(divs11.text.strip() if divs11 else 'Not Given')
        if 'Item Weight' in div.text.strip():
            divs12 = div.find_next_sibling('td', class_='a-size-base prodDetAttrValue')
            weight = clean_text(divs12.text.strip() if divs12 else 'Not Given')

    divs2a = soup.find('div', class_='a-popover-preload', id='a-popover-agShipMsgPopover')
    if (divs2a):
        table = divs2a.find('table', class_='a-lineitem')
        if (table):
            lines = table.find_all('td', class_='a-span9 a-text-left')
            for td in lines:
                if 'Price' in td.text:
                    divsprice = td.find_next_sibling('td', class_='a-span2 a-text-right')
                    price = clean_text(divsprice.text.strip() if divsprice else 'None')
                if 'AmazonGlobal Shipping' in td.text:
                    divsamzship = td.find_next_sibling('td', class_='a-span2 a-text-right')
                    priceamzship = clean_text(divsamzship.text.strip() if divsamzship else 'None')
                if 'Estimated Import Charges' in td.text:
                    diveic = td.find_next_sibling('td', class_='a-span2 a-text-right')
                    eic = clean_text(diveic.text.strip() if diveic else 'None')

    estimated_delivery_date_tag = soup.find('span', class_='a-text-bold')
    estimated_delivery_date = clean_text(estimated_delivery_date_tag.text.strip() if estimated_delivery_date_tag else 'N/A')
    crawl_date = datetime.datetime.now().strftime('%A, %B %d')
    shipping_days = calculate_shipping_days(estimated_delivery_date, crawl_date)
    
    return {
        "Dimension": dimension,
        "ASIN": asinid,
        "Date First Available": datefirstavailable,
        "Manufacturer": manufactures,
        "Country of Origin": country,
        "Best Sellers Rank": sellerrank,
        "Color": color,
        "Item Model Number": modelnumber,
        "Item Weight": weight,
        "Price": price,
        "AmazonGlobal Shipping": priceamzship,
        "Estimated Import Charges": eic,
        "Day Delivered": str(shipping_days)
    }

In [13]:
url_list = pd.read_csv(f'part1_data_{main_category}_{sub_category}.csv')


In [14]:
results = []

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3")
chrome_options.add_argument("--log-level=3")

driver = webdriver.Chrome(options=chrome_options)

In [15]:
retry_limit = 3  # Đặt giới hạn số lần thử lại

count = 0
for urldetail in url_list['Link']:
    count += 1
    retry_count = 0
    success = False
    
    print(f"Lần lấy thông tin thứ: {count}")

    while retry_count < retry_limit and not success:
        try:
            driver.get(urldetail)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, 'prodDetails')) or 
                EC.presence_of_element_located((By.ID, 'detailBulletsWrapper_feature_div'))
            )
            success = True
        except TimeoutException:
            retry_count += 1
            print(f"Timeout while waiting for page to load: {urldetail}. Retrying {retry_count}/{retry_limit}")
            time.sleep(5)  # Chờ 5 giây trước khi thử lại

    if not success:
        print(f"Failed to load page after {retry_limit} attempts: {urldetail}")
        continue

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    divs2 = soup.find('div', id='prodDetails')
    divs2I = soup.find('div', id='detailBulletsWrapper_feature_div')

    if divs2:
        result = detailproduct_table(soup, divs2)
    elif divs2I:
        result = detailproduct_bucket(soup, divs2I)
    
    if result:
        results.append(result)
    
    print(result)

driver.quit()

In [16]:
filename = f"part2_dataDetail_{main_category}_{sub_category}.csv"
with open(filename, 'w', newline="", encoding='utf-8') as csv_file:
    csv_writer = csv.DictWriter(csv_file, fieldnames=result.keys())
    csv_writer.writeheader()
    csv_writer.writerows(results)

print(f"Completed scraping and saved to {filename}.")

NameError: name 'result' is not defined